In [ ]:
# 1. 알고(구현): 백준 2564 (경비원) - https://www.acmicpc.net/problem/2564

import sys

from io import StringIO
sys.stdin = StringIO('''10 5
3
1 4
3 2
2 8
2 3''')

input = sys.stdin.readline


def solve():
    X, Y = map(int, input().split())
    N = int(input())

    positions = [tuple(map(int, input().split())) for _ in range(N + 1)]

    x_y = []

    # 좌표로 변환 (서남 쪽 (0,0) 동북 쪽 (X, Y))
    for d, length in positions:

        if d == 1:  # 북쪽
            x_y.append((length, Y))
        elif d == 2:  # 남쪽
            x_y.append((length, 0))
        elif d == 3:  # 서쪽
            x_y.append((0, Y-length))
        else:
            x_y.append((X, Y-length))

    print(x_y)

    edges = []

    for x in range(X+1):
        edges.append((x, 0))
    for y in range(Y+1):
        edges.append((X, y))
    for x in range(X, -1, -1):
        edges.append((x, Y))
    for y in range(Y, -1, -1):
        edges.append((0, y))

    edge_length = len(edges)

    indexs = []
    for x, y in x_y:
        for j in range(edge_length):
            if (x, y) == edges[j]:
                indexs.append(j)

    print(indexs)

    

solve()

[(4, 5), (0, 3), (8, 0), (3, 0)]
[23, 30, 8, 3]


In [ ]:
# 2. 알고(수학): 백준 1358 (하키) - https://www.acmicpc.net/problem/1358
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 3. 알고(Map): 백준 20920 (영단어 암기는 괴로워) - https://www.acmicpc.net/problem/20920
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 4. 알고(그리디): 백준 1343 (폴리오미노) - https://www.acmicpc.net/problem/1343
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


```sql
-- 5. SQL(Lv.1): 모든 레코드 조회하기 - https://school.programmers.co.kr/learn/courses/30/lessons/59034
```

In [ ]:
# 1. 알고(시뮬): 백준 15683 (감시) - https://www.acmicpc.net/problem/15683
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 2. 알고(DP): 백준 1535 (안녕) - https://www.acmicpc.net/problem/1535
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 3. 알고(그래프): 백준 4485 (녹색 옷 입은 애가 젤다지?) - https://www.acmicpc.net/problem/4485
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


In [ ]:
# 4. 알고(백트래킹): 백준 1987 (알파벳) - https://www.acmicpc.net/problem/1987
import sys

from io import StringIO
sys.stdin = StringIO('''''')

input = sys.stdin.readline


def solve():

    pass


solve()


```sql
-- 5. SQL(Lv.3): 조회수가 가장 많은 중고거래 게시판의 첨부파일 조회하기 - https://school.programmers.co.kr/learn/courses/30/lessons/164671
```